In [1]:
import importlib

import pandas as pd
import optuna
import gc
import copy

import sys
sys.path.append("../nlp_assist/")

import pandas as pd
import optuna
import gc
import copy

#条件もとの3テーブルを作るモジュール
import nlp_connect as nl
importlib.reload(nl)

#各種条件をquboとして作るためのモジュール
import nlp_limiter as lmt
importlib.reload(lmt)

#並列で探索するソルバのラッパー
import p_solver as ps
importlib.reload(ps)

#結果チェッカ等
import res_parser as rps
importlib.reload(rps)

import nsp_settings as ns
importlib.reload(ns)

print("reloaded!")

import datetime

import pickle


dummy var generator activated
start status is 0
limter activated!
dummy var generator activated
start status is 0
limter activated!
reloaded!


In [2]:
dt_now = datetime.datetime.now()
dt_now.strftime('%Y%m%d_%H%M%S_%S')

'20230416_165205_05'

In [3]:
f = "../demos/dmk0.3_20220111_18.xlsx"
f2 = "../demos/dmk0.3_20220111_18add.xlsx"
# x = nl.xlsx2condtion(f)
# x.make3CDT()

# # q_genはqubo generator
# qg = lmt.q_gen(x2c=x)

# nst = ns.nsp_settings()
# inclist = nst.check_list
# inclist

In [4]:
ns1 = ns.nsp_settings.param_dict.copy()
set(ns1.keys())

{'1by1',
 'cnight',
 'cwork',
 'dab',
 'eqg2',
 'eqg3',
 'fseq',
 'lcnn',
 'lcnt',
 'lfbd',
 'lnd',
 'mps',
 'nnv',
 'nseq',
 'rqh',
 'rqn',
 'sc',
 'tn',
 'tt'}

In [5]:

opt_dict = dict()
# opt_dict[0] = set(ns1.keys())
opt_dict[0] = ('1by1','cnight',
     'cwork',
     'dab',
    #  'eqg2',
    #  'eqg3',
     'fseq',
     'lcnn',
     'lcnt',
     'lfbd',
     'lnd',
     'mps',
     'nnv',
     'nseq',
     'rqh',
     'rqn',
     'sc',
     'tn',
     'tt')
print(len(opt_dict[0]))
# opt_dict[1] = ()
# opt_dict[2] = ()
# opt_dict[3] = ( 'eqg3', 'eqg2')


17


In [6]:
# d1 = ns.nsp_settings.param_dict.copy()
# d1
# from multiprocessing import Manager
# from multiprocessing import Pool
# from collections import namedtuple

In [7]:
opt_dict[0]

('1by1',
 'cnight',
 'cwork',
 'dab',
 'fseq',
 'lcnn',
 'lcnt',
 'lfbd',
 'lnd',
 'mps',
 'nnv',
 'nseq',
 'rqh',
 'rqn',
 'sc',
 'tn',
 'tt')

In [8]:
v_l = []
v_l = v_l + list(opt_dict[0])
print(v_l)

inc = v_l
d1 = ns.nsp_settings.param_dict.copy()
for t in inc:
    d1[t] = 1
    
x = nl.xlsx2condtion(f)
x.make3CDT(year=2020, month=10)

# q_genはqubo generator
qg = lmt.q_gen(x2c=x)
#qg2  = copy.deepcopy(qg)
qg.setParam(param_dict=d1)
# for i in inc:
#     print(i)
#     qg.makeAllLimitation(do_normalize=False, hideProgressBar=True, include=[i])
#     qdf = qg.ne.getQ_asDF()
#     print(qdf.head())

qg.makeAllLimitation(do_normalize=False, hideProgressBar=True, include=inc)
qdf = qg.ne.getQ_asDF()


['1by1', 'cnight', 'cwork', 'dab', 'fseq', 'lcnn', 'lcnt', 'lfbd', 'lnd', 'mps', 'nnv', 'nseq', 'rqh', 'rqn', 'sc', 'tn', 'tt']
xlsx file is
../demos/dmk0.3_20220111_18.xlsx
make condition tables.
holidaymode: asHoliday
year:  2020
month:  10
holidaymode:asHoliday
days that is holiday and not sunday will be Holiday.
Q generator activated!
x2c need to pre make 3DT
nsp_expression activated!
Q dict is resetted!
one can do one task per day...
limit continuous work by dt1 and member capacity
limit continious night work by dt3...
forbiden sequence is forbiden..
must pair must nor...
limitting as req numbers
limitting as req work times
limit total tasks...
limit tatal nights...
last month info: duty class to forbiden seq...
last month info: duty class to need seq...
last month info: cont work...
last month info: cont night work...
shift count quantified...
limit member pair shift...
Veteran in each group should be preserverd.
Night time novis should not be two in group.


100%|██████████████████████████████████| 579726/579726 [00:00<00:00, 4604409.24it/s]


In [13]:
import joblib
from joblib import Parallel, delayed
from time import time

In [15]:
import random

In [16]:
random.randint(1, 1000000)

215445

In [ ]:
%%time
for i in range(1):
    # print(opt_dict[i])
    v_l = []
    for j in range(i +1):
        v_l = v_l + list(opt_dict[j])
    print(i, j, v_l)
    
    inc = v_l
    
    def objective_i(trial):        
        d1 = ns.nsp_settings.param_dict.copy()
        
        for t in inc:
            d1[t] = trial.suggest_int(t, 1, 256)
            # d1[t] = trial.suggest_float(t, 1, 256)
    
        
        #sweeps_log = (trial.suggest_float('sweeps_lg', 1, 10))
        #sweeps = int(2 ** sweeps_log)
        swmul = (trial.suggest_int('sweeps', 10, 1000))
        #pppw = 10 **i
        
        # dummyseed = (trial.suggest_int('dummyseed', 1, 100000))
        dummyseed = random.randint(1, 1000000)
        #sw = int(sweeps * 1000)
#         sw = 1000000
        sw_base = 1e+4
        sw = int(sw_base *swmul)
        # sw = int(sweeps * 1000 * pppw)

        x = nl.xlsx2condtion(f)
        x.make3CDT(year=2020, month=10)

        # q_genはqubo generator
        qg = lmt.q_gen(x2c=x)
        #qg2  = copy.deepcopy(qg)
        qg.setParam(param_dict=d1)
        qg.makeAllLimitation(do_normalize=True, hideProgressBar=True, include=inc)

        p = ps.psol(q=qg.ne.getQ(),
                 num_reads=1,
                 sweeps=sw,
                   seed=dummyseed
                   )
        mins = p.p_solve_getMin(threads=5, times=5)
        
        rp = rps.res_parser(x=x)
        rp.prepRes(res=mins.sample, hideProgressBar=True)
        rc = rps.res_checker(x=x, echo_bool=True)
        rc.checkAllLimitation(res_parser=rp, include=inc)
        colp_num = len(rc.log_df)
        score = colp_num
#         score = (colp_num + 0.0000001) * sweeps_log

        if score <= 2:
            res_dict = dict()
            res_dict["mins_sample"] = mins.sample
            tm = rps.table_maker(x=x, res=mins.sample)
            res_dict["dummyseed"] = dummyseed
            res_dict["log_df"] = rc.log_df
            res_dict["timetable"]= tm.datetable.copy()
            res_dict["shifttable"] = tm.shifttable.copy()
            dt_now = datetime.datetime.now()
            ts = dt_now.strftime('%Y%m%d_%H%M%S_%f')
            f_pkl = "./res_dict_"+"error"+str(score) +"_"+ts+".pkl"
            # f_pkl
            with open(f_pkl, "wb") as fx:
                pickle.dump(res_dict, fx)

        # score = len(rc.log_df)
        print("colappse:" +str(colp_num))
        print('score:' + str(score))
        gc.collect()

        return score
    
    study_name = 'nsp-study_dummy4create'    
    storage_file = 'sqlite:///./optuna_study_itr_whole_fix_lchs.db'
    optuna.create_study(study_name=study_name,
                                    storage=storage_file,
                                    load_if_exists=True,
                                    sampler=optuna.samplers.TPESampler(seed=114514))
    
#     n_trails = 1
    n_trails = 1e+5
#     n_trais = 1000*(i*i +1)
    
    def s_opt(objective, iter_num, n_trials=n_trails,  gc_after_trial=True):
        print("ntraials:" + str(n_trials))
        # study_name = 'nsp-study_i_' + str(i)
        study_name = 'nsp-study_deg'
        #storage_file = 'sqlite:///./optuna_study_itr_' + str(i)+".db"
        #print(study_name, storage_file)
        study = optuna.create_study(study_name=study_name,
                                    storage=storage_file,
                                    load_if_exists=True,
                                    sampler=optuna.samplers.TPESampler(seed=114514 +iter_num))
        study.optimize(objective, n_trials=n_trials,  gc_after_trial=gc_after_trial)
    

    threads = 3
    obj_l = [(copy.deepcopy(objective_i), t) for t in range(threads)]
    print(obj_l)
    r = Parallel(n_jobs=threads)( [delayed(s_opt)(objective=t[0], iter_num= t[1]) for t in obj_l] )
    print("done!")


[I 2023-04-16 16:52:15,903] Using an existing study with name 'nsp-study_dummy4create' instead of creating a new one.


0 0 ['1by1', 'cnight', 'cwork', 'dab', 'fseq', 'lcnn', 'lcnt', 'lfbd', 'lnd', 'mps', 'nnv', 'nseq', 'rqh', 'rqn', 'sc', 'tn', 'tt']
[(<function objective_i at 0x000001ACD8829EE0>, 0), (<function objective_i at 0x000001ACD8829EE0>, 1), (<function objective_i at 0x000001ACD8829EE0>, 2)]


In [ ]:
1